# Desafio do Titanic

O problema do Titanic é um clássico em ciência de dados, com o objetivo de prever a sobrevivência de passageiros em um naufrágio. É um problema de classificação binária baseado em um conjunto de dados com informações de 891 passageiros, onde 342 sobreviveram.

A base está disponível no github da Suzana Viana Mota, o link é https://github.com/suzanasvm/TitanicUdacity contei a base utilizada de 59.8 KB.

As atribuições da base são: PassengerId, que é a identificação dos passageiros; Survived, que indica se o passageiro sobreviveu (1) ou não (0); Pclass, que representa a classe; Name, que é o nome dos passageiros; Sex, que é o gênero; Age, que é a idade em anos; SibSp, que representa a quantidade de irmãos ou cônjuges a bordo do Titanic; Parch, que representa a quantidade de pais ou crianças a bordo do Titanic; Ticket, que é o número do bilhete; Fare, que é a tarifa paga; Cabin, que é a cabine; e Embarked, que indica o local do embarque.

In [51]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pandas  as pd

In [29]:
base_original = pd.read_csv("titanic-data-6.csv")

In [30]:
base_original[["nome","sobrenome"]] = base_original["Name"].str.split(",", expand=True)

In [31]:
base_original["sobrenome"].str.split(".", expand=True)

,0,1,2
0,Mr,Owen Harris,None
1,Mrs,John Bradley (Florence Briggs Thayer),None
2,Miss,Laina,None
3,Mrs,Jacques Heath (Lily May Peel),None
4,Mr,William Henry,None
...,...,...,...
886,Rev,Juozas,None
887,Miss,Margaret Edith,None
888,Miss,"Catherine Helen ""Carrie""",None
889,Mr,Karl Howell,None


In [32]:
base_original[["titulos","sobrenome", "sujeira"]] = base_original["sobrenome"].str.split(".",    expand=True)

In [37]:
base_original.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'nome', 'sobrenome',
       'titulos', 'sujeira'],
      dtype='object')

In [38]:
base_analisada = base_original[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked',
       'titulos']]


In [44]:
base_analisada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
 11  titulos      891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [73]:
base_analisada["Ticket"].dtypes == "O"

True

In [75]:
base_analisada_str = pd.DataFrame()
base_analisada_int = pd.DataFrame()

In [76]:
for i in base_analisada.columns:
    if base_analisada[i].dtypes == "O":
        base_analisada_str[i] = base_analisada[i]
    else:
        base_analisada_int[i] = base_analisada[i]


In [78]:
base_analisada_str

,Sex,Ticket,Cabin,Embarked,titulos
0,male,A/5 21171,NaN,S,Mr
1,female,PC 17599,C85,C,Mrs
2,female,STON/O2. 3101282,NaN,S,Miss
3,female,113803,C123,S,Mrs
4,male,373450,NaN,S,Mr
...,...,...,...,...,...
886,male,211536,NaN,S,Rev
887,female,112053,B42,S,Miss
888,female,W./C. 6607,NaN,S,Miss
889,male,111369,C148,C,Mr


In [46]:
base_analisada.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [55]:
base_analisada.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'titulos'],
      dtype='object')

In [64]:
from sklearn.model_selection import train_test_split
import numpy as np

X = base_analisada[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'titulos']].values
y = base_analisada["Survived"].values

# Separa os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Imprime o número de amostras em cada conjunto
print(f"Treinamento: {len(X_train)}, Teste: {len(X_test)}")

Treinamento: 623, Teste: 268


In [65]:
dt_train = pd.DataFrame(X_train, columns= base_analisada.columns)

In [87]:
base_analisada.Survived.value_counts()/len(base_analisada.Survived)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [88]:
dt_train.Survived.value_counts()/len(dt_train.Survived)

0    0.629213
1    0.370787
Name: Survived, dtype: float64